In [1]:
# Run this block first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Heating tutorial 4

<img src="figures/ecs3.png" alt="heating" width="800"/>

A family of 4 uses 150 litres of domestic hot water (DHW) at 50°C per day. Their single-family house has a heat loss coefficient of 120 W/K. They are located in Chambéry, where the minimal temperature for sizing heating systems is -11°C and the total of heating degree days is 2,700 °C.d annually. We want to propose a solution for a combined solar system producing DHW and space heating.

### 1. Estimating the consumption

Estimate the total annual needs for space heating and DHW, in kWh. We suppose that cold water comes at 10°C.

In [2]:
# Exercise data
daily_consumption = 150
T_hot = 50
T_cold = 10
T_min = -11
H = 120
HDD = 2700

# question 1: estimating the yearly consumption (kWh)
E_sh = H * HDD * 24 / 1000
E_dhw = daily_consumption * 4180 * (T_hot-T_cold) / (3.6e6) * 365

print("Consumption for space heating (kWh): ", E_sh)
print("Consumption for domestic hot water (kWh): ", E_dhw)

Consumption for space heating (kWh):  7776.0
Consumption for domestic hot water (kWh):  2542.8333333333335


### 2. Sizing the system

Choose the surface of collectors and volume of storage.
* The suggested surface of collectors is 1 m$^2$ for each 1,000 kWh of total annual need (1 m$^2$ of collectors receives about 1,500 kWh of solar irradiance annually. With an efficiency of 30\%, 1 m$^2$ can then produce about 450 kWh a year)
* The suggested storage volume depends on the climate zone: in this climate, we suggest a volume of 67 litres per m$^2$ of collectors, for combined heating and DHW production

Calculate the energy capacity of the tank, if water is stored at 60°C.

In [3]:
# Total consumption
E_total = E_sh + E_dhw
# surface of collectors and volume of storage
surface = round(E_total/1000)
volume = 67 * surface
# energy capacity of the tank (kWh)
capacity = volume * 4180 * (60-T_cold) / (3.6e6)

print("Surface of collectors (m2): ", surface)
print("Storage volume (litres): ", volume)
print("Storage capacity (kWh): ", capacity)

Surface of collectors (m2):  10
Storage volume (litres):  670
Storage capacity (kWh):  38.897222222222226


### 3. Solar coverage

The efficiency of thermal collectors is highest when solar irradiance $E$ is high and the temperature difference $\Delta T$ between water and outdoor is low. For simplification, we will assume that the efficiency of the supply loop, from collectors to storage, is 40$\%$.

The following table gives the average outdoor temperature and total solar irradiance during a few typical days.

| Average temperature (°C) | Solar irradiance (kWh/m2/day) |
|---|---|
| -1 | 1.2 |
| 7 | 2.6 |
| 26 | 4.4 |

For each of these days, calculate: the space heating need; the solar production; the solar coverage for DHW and space heating, and the energy required from the auxiliary heating system. We will assume a base temperature for heating of 15°C.

In [4]:
T_base = 15
T_average = np.array([-1, 7, 26])
irradiance = np.array([1.2, 2.6, 4.4])

# heating and DHW need
deltaT = T_base - T_average
deltaT[deltaT < 0] = 0
heating_needs = H * deltaT * 24 / 1000
dhw_needs = daily_consumption * 4180 * (T_hot-T_cold) / (3.6e6)

# solar production
solar_prod = irradiance * surface * 0.4
solar_coverage = solar_prod / (heating_needs + dhw_needs)

print("Solar coverage (%): ", solar_coverage*100)

Solar coverage (%):  [  9.04863642  34.65896467 252.63157895]


# Hydraulic balancing

<img src="figures/heating_1.png" alt="heating1" width="500"/>

A network of radiators is set up so that each radiator is connected to a common collector through polypropylene pipes. The radiators operate in a lower temperature mode: 45/38/20. The heating power of all rooms are summarised here:

<img src="figures/heating_2.png" alt="heating2" width="700"/>

1. Estimate the flow rates circulating inside each radiator.
2. Choose the diameter of the pipes, so that the speed of water is near 0.3 m/s
3. Calculate the pressure loss of each loop. Which is the most penalized loop?

### Solution

The following block calculates the volumetric flow rates:

In [5]:
# Temperatures
T_in = 45
T_out = 38
T_room = 20

# Properties of the radiators
power = np.array([600, 400, 600, 600, 900])
length = np.array([7, 6, 16, 20, 20])

# Flow rates
mdot = power / (4180*(T_in-T_out))   # in kg/s
qdot = mdot * 3600 / 998

print("Flow rates in kg/s: ", mdot)
print("Flow rates in m3/h: ", qdot)

Flow rates in kg/s:  [0.02050581 0.01367054 0.02050581 0.02050581 0.03075871]
Flow rates in m3/h:  [0.07396885 0.04931257 0.07396885 0.07396885 0.11095328]


The flow rates are rather low compared to the values available on the chart (p. 5 on the handout). We will choose the diameters so that the speed of water is close to 0.3 m/s. Then, we can read the pressure loss (in Pa/m) on the left axis.

In [6]:
diameters = np.array([10, 8, 10, 10, 10])
# Coefficient of pressure loss in Pa/m
dp_coeff = np.array([10, 15, 10, 10, 20]) * 10
# Total pressure loss of the loop in Pa
dp_pipes = dp_coeff * length

Then we add a fully open valve to each of the 5 radiators. The pressure loss $\Delta p$ caused by a valve with a coefficient $K_v$ is given by the following formula:
$$ Q = K_v \sqrt{\frac{\Delta p}{d}} $$
where the flow rate $Q$ is in m$^3/h$, the pressure drop $\Delta p$ in bar, and the density $d$ of water is 1. The coefficient of a fully opened valve is 0.7.

In [7]:
K_v = 0.7 * np.ones(5)
dp_valve = (qdot/K_v)**2 * 1e5

print("Pressure loss in pipes (Pa): ", dp_pipes)
print("Pressure loss in valves (Pa): ", dp_valve)
print("Total (Pa): ", dp_pipes+dp_valve)

Pressure loss in pipes (Pa):  [ 700  900 1600 2000 4000]
Pressure loss in valves (Pa):  [1116.61047099  496.27132044 1116.61047099 1116.61047099 2512.37355973]
Total (Pa):  [1816.61047099 1396.27132044 2716.61047099 3116.61047099 6512.37355973]


Now we search the value of $K_v$ with which all radiators will have the same total pressure loss as the most penalized one.

In [8]:
# Most penalized radiator:
dp_max = np.max(dp_pipes+dp_valve)
# New value of the valve pressure loss that should be reached
dp_valve = dp_max - dp_pipes
# New value of K_v
K_v = qdot / np.sqrt(dp_valve / 1e5)

print("New coefficient K_v: ", K_v)

New coefficient K_v:  [0.30681185 0.2081538  0.33373611 0.34821411 0.7       ]
